# Day 4: Ceres Search

"Looks like the Chief's not here. Next!" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the Ceres monitoring station!

As the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she'd like to know if you could help her with her word search (your puzzle input). She only has to find one word: **XMAS**.

---

## The Word Search Rules

This word search allows words to be:
- Horizontal
- Vertical
- Diagonal
- Written backwards
- Overlapping other words

However, you don't merely need to find one instance of **XMAS** — you need to find **all of them**.

Here are a few ways **XMAS** might appear, where irrelevant characters have been replaced with `.`:

```
..X...
.SAMX.
.A..A.
XMAS.S
.X....
```

---

## Example Word Search

Here is an example of a word search:

```
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX
```

### Total Matches
In this word search, **XMAS** occurs a total of **18 times**. Below is the same word search with letters not involved in any **XMAS** replaced with `.`:

```
....XXMAS.
.SAMXMS...
...S..A...
..A.A.MS.X
XMASAMX.MM
X.....XA.A
S.S.S.S.SS
.A.A.A.A.A
..M.M.M.MM
.X.X.XMASX
```

---

## Task

Take a look at the little Elf's word search. **How many times does XMAS appear?**

In [1]:
# target word is XMAS
# loop over the word search puzzle and find the first letter of the target word
# then check if the next letter is in the adjacent cells
# if the next letter is found, continue to the next letter
# if the next letter is not found, break the loop and continue to the next cell
# if the target word is found, increment the count of the target word
from enum import Enum
import numpy as np
import numpy.typing as npt


class Direction(Enum):
    UP = [[0, 1, 0], [0, 0, 0], [0, 0, 0]]
    DOWN = [[0, 0, 0], [0, 0, 0], [0, 1, 0]]
    LEFT = [[0, 0, 0], [1, 0, 0], [0, 0, 0]]
    RIGHT = [[0, 0, 0], [0, 0, 1], [0, 0, 0]]
    UP_LEFT = [[1, 0, 0], [0, 0, 0], [0, 0, 0]]
    UP_RIGHT = [[0, 0, 1], [0, 0, 0], [0, 0, 0]]
    DOWN_LEFT = [[0, 0, 0], [0, 0, 0], [1, 0, 0]]
    DOWN_RIGHT = [[0, 0, 0], [0, 0, 0], [0, 0, 1]]

    @property
    def mask(self) -> npt.NDArray[np.bool_]:
        """Convert the direction's matrix to a NumPy boolean array."""
        return np.array(self.value, dtype=bool)

    @property
    def dx(self) -> int:
        """Get the x-coordinate of the direction."""
        return np.nonzero(self.mask)[0][0] - 1

    @property
    def dy(self) -> int:
        """Get the y-coordinate of the direction."""
        return np.nonzero(self.mask)[1][0] - 1


def get_directions(roi: np.ndarray[np.bool_]):
    assert roi.shape == (3, 3)
    for direction in Direction:
        if np.array_equal(roi * direction.mask, direction.mask):
            yield direction


def get_neighbourhood_roi(grid, i, j):
    return grid[i - 1 : i + 2, j - 1 : j + 2]


def word_search_recursive(
    grid: np.ndarray, word: str, x: int, y: int, direction: Direction
):
    if not word:
        return True  # base case
    next_x, next_y = x + direction.dx, y + direction.dy
    if grid[next_x, next_y] == word[0]:
        return word_search_recursive(grid, word[1:], next_x, next_y, direction)


def word_search(grid: np.ndarray, word: str):
    count = 0
    # pad to handle edge cases
    grid = np.pad(grid, 1, constant_values=" ")
    starting_points = np.nonzero(grid == word[0])
    for x, y in np.column_stack(starting_points):
        roi = get_neighbourhood_roi(grid, x, y)
        for direction in get_directions(roi == word[1]):
            if word_search_recursive(grid, word[1:], x, y, direction):
                count += 1
    return count

grid = np.array([list(row) for row in np.loadtxt("./example.txt", dtype=str)])
print(f"{word_search(grid, "XMAS")=}")

grid = np.array([list(row) for row in np.loadtxt("./input.txt", dtype=str)])
print(f"{word_search(grid, "XMAS")=}")

word_search(grid, "XMAS")=18
word_search(grid, "XMAS")=2464


In [2]:
# correlation approach
# create a list of kernels for the target word and its rotations
# then correlate the grid with each kernel
# if the maximum correlation is found, the target word is found
# increment the count of the target word
import numpy as np
from scipy.signal import correlate2d

def get_kernels_for_word(word: str) -> list[np.ndarray]:
    filters = []
    chars = np.array(list(word))
    # horizontal
    filters.append(chars.reshape(1, -1))
    filters.append(chars[::-1].reshape(1, -1))
    # vertical
    filters.append(chars.reshape(-1, 1))
    filters.append(chars[::-1].reshape(-1, 1))
    # diagonal (at 0°, 45°, 90°, 135°)
    for i in range(4):
        filters.append(np.rot90(np.diag(chars), i))
    return filters

def word_search_with_correlation(grid: np.ndarray, kernels: list[np.ndarray]) -> int:
    # convert the grid and kernels to numbers for correlation to work
    char_to_num = np.vectorize(lambda c: ord(c) if c != "" else 0)
    kernels = [char_to_num(kernel) for kernel in kernels]
    grid = char_to_num(grid)

    count = 0
    for kernel in kernels:
        max_correlation = correlate2d(kernel, kernel, mode="valid").max()
        count += (correlate2d(grid, kernel, mode="valid") == max_correlation).sum()
    return count

for kernel in get_kernels_for_word("XMAS"):
    print(kernel)

grid = np.array([list(row) for row in np.loadtxt("./example.txt", dtype=str)])
print(f"{word_search_with_correlation(grid, get_kernels_for_word("XMAS"))=}")

grid = np.array([list(row) for row in np.loadtxt("./input.txt", dtype=str)])
print(f"{word_search_with_correlation(grid, get_kernels_for_word("XMAS"))=}")

[['X' 'M' 'A' 'S']]
[['S' 'A' 'M' 'X']]
[['X']
 ['M']
 ['A']
 ['S']]
[['S']
 ['A']
 ['M']
 ['X']]
[['X' '' '' '']
 ['' 'M' '' '']
 ['' '' 'A' '']
 ['' '' '' 'S']]
[['' '' '' 'S']
 ['' '' 'A' '']
 ['' 'M' '' '']
 ['X' '' '' '']]
[['S' '' '' '']
 ['' 'A' '' '']
 ['' '' 'M' '']
 ['' '' '' 'X']]
[['' '' '' 'X']
 ['' '' 'M' '']
 ['' 'A' '' '']
 ['S' '' '' '']]
word_search_with_correlation(grid, get_kernels_for_word("XMAS"))=18
word_search_with_correlation(grid, get_kernels_for_word("XMAS"))=2464


# Part Two

The Elf looks quizzically at you. Did you misunderstand the assignment?

Looking for the instructions, you flip over the word search to find that this isn't actually an **XMAS** puzzle; it's an **X-MAS** puzzle in which you're supposed to find **two MAS in the shape of an X**. One way to achieve that is like this:

```
M.S
.A.
M.S
```

Irrelevant characters have again been replaced with `.` in the above diagram. Within the X, each **MAS** can be written forwards or backwards.

---

## Updated Example

Here's the same example from before, but this time all of the **X-MAS** shapes have been kept instead:

```
.M.S......
..A..MSMS.
.M.S.MAA..
..A.ASMSM.
.M.S.M....
..........
S.S.S.S.S.
.A.A.A.A..
M.M.M.M.M.
..........
```

### Total Count

In this example, an **X-MAS** appears **9 times**.

---

## Task

Flip the word search from the instructions back over to the word search side and try again. **How many times does an X-MAS appear?**

In [3]:
def get_kernels_for_x_shape(word: str):
    kernels = []
    cross = np.array([["M", "", "S"], ["", "A", ""], ["M", "", "S"]])
    for rot in range(4):
        kernels.append(np.rot90(cross, rot))
    return kernels


for kernel in get_kernels_for_x_shape("MAS"):
    print(kernel)

grid = np.array([list(row) for row in np.loadtxt("./example.txt", dtype=str)])
print(f"{word_search_with_correlation(grid, get_kernels_for_x_shape("MAS"))=}")

grid = np.array([list(row) for row in np.loadtxt("./input.txt", dtype=str)])
print(f"{word_search_with_correlation(grid, get_kernels_for_x_shape("MAS"))=}")

[['M' '' 'S']
 ['' 'A' '']
 ['M' '' 'S']]
[['S' '' 'S']
 ['' 'A' '']
 ['M' '' 'M']]
[['S' '' 'M']
 ['' 'A' '']
 ['S' '' 'M']]
[['M' '' 'M']
 ['' 'A' '']
 ['S' '' 'S']]
word_search_with_correlation(grid, get_kernels_for_x_shape("MAS"))=9
word_search_with_correlation(grid, get_kernels_for_x_shape("MAS"))=1982
